# Taxi Env (Don't Modify it )

In [2]:
!pip install gymnasium

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from contextlib import closing
from io import StringIO
from os import path

import numpy as np

import gymnasium as gym
from gymnasium import Env, spaces, utils
from gymnasium.envs.toy_text.utils import categorical_sample
from gymnasium.error import DependencyNotInstalled

MAP_0 = [
    "+---------+",
    "|R: | : :G|",
    "| : | : : |",
    "| : : : : |",
    "| | : | : |",
    "|Y| : |B: |",
    "+---------+",
]

MAP_1 = [
    "+---------+",
    "|R| : : :G|",
    "| | : | : |",
    "| : : | : |",
    "| : | : : |",
    "|Y: | :B: |",
    "+---------+",
]

MAP_2 = [
    "+---------+",
    "|R: : : |G|",
    "| : : : | |",
    "| : | : : |",
    "| : | | : |",
    "|Y: : |B: |",
    "+---------+",
]

MAP = MAP_0
MAPS = [MAP_0, MAP_1, MAP_2]
WINDOW_SIZE = (550, 350)

In [ ]:
class TaxiEnv(Env):

    metadata = {
        "render_modes": ["human", "ansi", "rgb_array"],
        "render_fps": 4,
    }

    def __init__(
        self,
        render_mode: str | None = None,
        is_rainy: bool = True,
        fickle_passenger: bool = False,
        use_multiple_maps: bool = True,
        reward_step: float = -1,
        reward_delivery: float = 20,
        reward_illegal: float = -10,
    ):
        self.use_multiple_maps = use_multiple_maps

        if use_multiple_maps:
            self.desc_maps = [np.asarray(m, dtype="c") for m in MAPS]
            self.P_maps = []
            self.current_map_id = 0
            self.desc = self.desc_maps[0]
        else:
            self.desc = np.asarray(MAP, dtype="c")

        self.locs = locs = [(0, 0), (0, 4), (4, 0), (4, 3)]
        self.locs_colors = [(255, 0, 0), (0, 255, 0), (255, 255, 0), (0, 0, 255)]

        self.reward_step = reward_step
        self.reward_delivery = reward_delivery
        self.reward_illegal = reward_illegal

        num_states = 500
        num_rows = 5
        num_columns = 5
        self.max_row = num_rows - 1
        self.max_col = num_columns - 1
        self.initial_state_distrib = np.zeros(num_states)
        num_actions = 6


        if use_multiple_maps:

            for map_idx in range(3):
                self.desc = self.desc_maps[map_idx]
                P = {
                    state: {action: [] for action in range(num_actions)}
                    for state in range(num_states)
                }
                self.P = P

                # Iterate through all possible state combinations
                for row in range(num_rows):
                    for col in range(num_columns):
                        for pass_idx in range(len(locs) + 1):
                            for dest_idx in range(len(locs)):
                                state = self.encode(row, col, pass_idx, dest_idx)
                                if pass_idx < 4 and pass_idx != dest_idx:
                                    self.initial_state_distrib[state] += 1
                                for action in range(num_actions):
                                    if is_rainy:
                                        self._build_rainy_transitions(
                                            row, col, pass_idx, dest_idx, action,
                                        )
                                    else:
                                        self._build_dry_transitions(
                                            row, col, pass_idx, dest_idx, action,
                                        )

                self.P_maps.append(P)


            self.desc = self.desc_maps[0]
            self.P = self.P_maps[0]
        else:

            self.P = {
                state: {action: [] for action in range(num_actions)}
                for state in range(num_states)
            }

            for row in range(num_rows):
                for col in range(num_columns):
                    for pass_idx in range(len(locs) + 1):
                        for dest_idx in range(len(locs)):
                            state = self.encode(row, col, pass_idx, dest_idx)
                            if pass_idx < 4 and pass_idx != dest_idx:
                                self.initial_state_distrib[state] += 1
                            for action in range(num_actions):
                                if is_rainy:
                                    self._build_rainy_transitions(
                                        row, col, pass_idx, dest_idx, action,
                                    )
                                else:
                                    self._build_dry_transitions(
                                        row, col, pass_idx, dest_idx, action,
                                    )

        self.initial_state_distrib /= self.initial_state_distrib.sum()
        self.action_space = spaces.Discrete(num_actions)
        self.observation_space = spaces.Discrete(num_states)

        self.render_mode = render_mode
        self.fickle_passenger = fickle_passenger
        self.fickle_step = self.fickle_passenger and self.np_random.random() < 0.5

        # pygame utils
        self.window = None
        self.clock = None
        self.cell_size = (
            WINDOW_SIZE[0] / self.desc.shape[1],
            WINDOW_SIZE[1] / self.desc.shape[0],
        )
        self.taxi_imgs = None
        self.taxi_orientation = 0
        self.passenger_img = None
        self.destination_img = None
        self.median_horiz = None
        self.median_vert = None
        self.background_img = None

    def _pickup(self, taxi_loc, pass_idx, reward):
        """Computes the new location and reward for pickup action."""
        if pass_idx < 4 and taxi_loc == self.locs[pass_idx]:
            new_pass_idx = 4
            new_reward = reward
        else:  # passenger not at location
            new_pass_idx = pass_idx
            new_reward = self.reward_illegal

        return new_pass_idx, new_reward

    def _dropoff(self, taxi_loc, pass_idx, dest_idx, default_reward):
        """Computes the new location and reward for return dropoff action."""
        if (taxi_loc == self.locs[dest_idx]) and pass_idx == 4:
            new_pass_idx = dest_idx
            new_terminated = True
            new_reward = self.reward_delivery
        elif (taxi_loc in self.locs) and pass_idx == 4:
            new_pass_idx = self.locs.index(taxi_loc)
            new_terminated = False
            new_reward = default_reward
        else:  # dropoff at wrong location
            new_pass_idx = pass_idx
            new_terminated = False
            new_reward = self.reward_illegal

        return new_pass_idx, new_reward, new_terminated

    def _calc_new_position(self, row, col, movement, offset=0):
        """Calculates the new position for a row and col to the movement."""
        dr, dc = movement
        new_row = max(0, min(row + dr, self.max_row))
        new_col = max(0, min(col + dc, self.max_col))
        if self.desc[1 + new_row, 2 * new_col + offset] == b":":
            return new_row, new_col
        else:  # Default to current position if not traversable
            return row, col

    def _build_rainy_transitions(self, row, col, pass_idx, dest_idx, action):
        """Computes the next action for a state (row, col, pass_idx, dest_idx) and action for `is_rainy`."""
        state = self.encode(row, col, pass_idx, dest_idx)

        taxi_loc = left_pos = right_pos = (row, col)
        new_row, new_col, new_pass_idx = row, col, pass_idx
        reward = self.reward_step
        terminated = False

        moves = {
            0: ((1, 0), (0, -1), (0, 1)),  # Down
            1: ((-1, 0), (0, -1), (0, 1)),  # Up
            2: ((0, 1), (1, 0), (-1, 0)),  # Right
            3: ((0, -1), (1, 0), (-1, 0)),  # Left
        }

        # Check if movement is allowed
        if (
            action in {0, 1}
            or (action == 2 and self.desc[1 + row, 2 * col + 2] == b":")
            or (action == 3 and self.desc[1 + row, 2 * col] == b":")
        ):
            dr, dc = moves[action][0]
            new_row = max(0, min(row + dr, self.max_row))
            new_col = max(0, min(col + dc, self.max_col))

            left_pos = self._calc_new_position(row, col, moves[action][1], offset=2)
            right_pos = self._calc_new_position(row, col, moves[action][2])
        elif action == 4:  # pickup
            new_pass_idx, reward = self._pickup(taxi_loc, new_pass_idx, reward)
        elif action == 5:  # dropoff
            new_pass_idx, reward, terminated = self._dropoff(
                taxi_loc, new_pass_idx, dest_idx, reward
            )
        intended_state = self.encode(new_row, new_col, new_pass_idx, dest_idx)

        if action <= 3:
            left_state = self.encode(left_pos[0], left_pos[1], new_pass_idx, dest_idx)
            right_state = self.encode(
                right_pos[0], right_pos[1], new_pass_idx, dest_idx
            )

            self.P[state][action].append((0.8, intended_state, self.reward_step, terminated))
            self.P[state][action].append((0.1, left_state, self.reward_step, terminated))
            self.P[state][action].append((0.1, right_state, self.reward_step, terminated))
        else:
            self.P[state][action].append((1.0, intended_state, reward, terminated))


    def encode(self, taxi_row, taxi_col, pass_loc, dest_idx):
        # (5) 5, 5, 4
        i = taxi_row
        i *= 5
        i += taxi_col
        i *= 5
        i += pass_loc
        i *= 4
        i += dest_idx
        return i

    def decode(self, i):
        out = []
        out.append(i % 4)
        i = i // 4
        out.append(i % 5)
        i = i // 5
        out.append(i % 5)
        i = i // 5
        out.append(i)
        assert 0 <= i < 5
        return reversed(out)

    def action_mask(self, state: int):
        """Computes an action mask for the action space using the state information."""
        mask = np.zeros(6, dtype=np.int8)
        taxi_row, taxi_col, pass_loc, dest_idx = self.decode(state)
        if taxi_row < 4:
            mask[0] = 1
        if taxi_row > 0:
            mask[1] = 1
        if taxi_col < 4 and self.desc[taxi_row + 1, 2 * taxi_col + 2] == b":":
            mask[2] = 1
        if taxi_col > 0 and self.desc[taxi_row + 1, 2 * taxi_col] == b":":
            mask[3] = 1
        if pass_loc < 4 and (taxi_row, taxi_col) == self.locs[pass_loc]:
            mask[4] = 1
        if pass_loc == 4 and (
            (taxi_row, taxi_col) == self.locs[dest_idx]
            or (taxi_row, taxi_col) in self.locs
        ):
            mask[5] = 1
        return mask

    def step(self, a):
        transitions = self.P[self.s][a]
        i = categorical_sample([t[0] for t in transitions], self.np_random)
        p, s, r, t = transitions[i]
        self.lastaction = a

        shadow_row, shadow_col, shadow_pass_loc, shadow_dest_idx = self.decode(self.s)
        taxi_row, taxi_col, pass_loc, _ = self.decode(s)

        # If we are in the fickle step, the passenger has been in the vehicle for at least a step and this step the
        # position changed
        if (
            self.fickle_passenger
            and self.fickle_step
            and shadow_pass_loc == 4
            and (taxi_row != shadow_row or taxi_col != shadow_col)
        ):
            self.fickle_step = False
            possible_destinations = [
                i for i in range(len(self.locs)) if i != shadow_dest_idx
            ]
            dest_idx = self.np_random.choice(possible_destinations)
            s = self.encode(taxi_row, taxi_col, pass_loc, dest_idx)

        self.s = s

        if self.render_mode == "human":
            self.render()
        # truncation=False as the time limit is handled by the `TimeLimit` wrapper added during `make`
        return int(s), r, t, False, {"prob": p, "action_mask": self.action_mask(s)}

    def reset(
        self,
        *,
        seed: int | None = None,
        options: dict | None = None,
    ):
        super().reset(seed=seed)


        if self.use_multiple_maps and seed is not None:
            map_id = seed % 3
            self.current_map_id = map_id
            self.desc = self.desc_maps[map_id]
            self.P = self.P_maps[map_id]

        self.s = categorical_sample(self.initial_state_distrib, self.np_random)
        self.lastaction = None
        self.fickle_step = self.fickle_passenger and self.np_random.random() < 0.3
        self.taxi_orientation = 0

        if self.render_mode == "human":
            self.render()
        return int(self.s), {"prob": 1.0, "action_mask": self.action_mask(self.s)}

    def render(self):
        if self.render_mode is None:
            assert self.spec is not None
            gym.logger.warn(
                "You are calling render method without specifying any render mode. "
                "You can specify the render_mode at initialization, "
                f'e.g. gym.make("{self.spec.id}", render_mode="rgb_array")'
            )
            return
        elif self.render_mode == "ansi":
            return self._render_text()
        else:  # self.render_mode in {"human", "rgb_array"}:
            return self._render_gui(self.render_mode)

    def _render_gui(self, mode):
        try:
            import pygame  # dependency to pygame only if rendering with human
        except ImportError as e:
            raise DependencyNotInstalled(
                'pygame is not installed, run `pip install "gymnasium[toy-text]"`'
            ) from e

        if self.window is None:
            pygame.init()
            pygame.display.set_caption("Taxi")
            if mode == "human":
                self.window = pygame.display.set_mode(WINDOW_SIZE)
            elif mode == "rgb_array":
                self.window = pygame.Surface(WINDOW_SIZE)

        assert (
            self.window is not None
        ), "Something went wrong with pygame. This should never happen."
        if self.clock is None:
            self.clock = pygame.time.Clock()
        if self.taxi_imgs is None:
            file_names = [
                path.join(path.dirname(__file__), "img/cab_front.png"),
                path.join(path.dirname(__file__), "img/cab_rear.png"),
                path.join(path.dirname(__file__), "img/cab_right.png"),
                path.join(path.dirname(__file__), "img/cab_left.png"),
            ]
            self.taxi_imgs = [
                pygame.transform.scale(pygame.image.load(file_name), self.cell_size)
                for file_name in file_names
            ]
        if self.passenger_img is None:
            file_name = path.join(path.dirname(__file__), "img/passenger.png")
            self.passenger_img = pygame.transform.scale(
                pygame.image.load(file_name), self.cell_size
            )
        if self.destination_img is None:
            file_name = path.join(path.dirname(__file__), "img/hotel.png")
            self.destination_img = pygame.transform.scale(
                pygame.image.load(file_name), self.cell_size
            )
            self.destination_img.set_alpha(170)
        if self.median_horiz is None:
            file_names = [
                path.join(path.dirname(__file__), "img/gridworld_median_left.png"),
                path.join(path.dirname(__file__), "img/gridworld_median_horiz.png"),
                path.join(path.dirname(__file__), "img/gridworld_median_right.png"),
            ]
            self.median_horiz = [
                pygame.transform.scale(pygame.image.load(file_name), self.cell_size)
                for file_name in file_names
            ]
        if self.median_vert is None:
            file_names = [
                path.join(path.dirname(__file__), "img/gridworld_median_top.png"),
                path.join(path.dirname(__file__), "img/gridworld_median_vert.png"),
                path.join(path.dirname(__file__), "img/gridworld_median_bottom.png"),
            ]
            self.median_vert = [
                pygame.transform.scale(pygame.image.load(file_name), self.cell_size)
                for file_name in file_names
            ]
        if self.background_img is None:
            file_name = path.join(path.dirname(__file__), "img/taxi_background.png")
            self.background_img = pygame.transform.scale(
                pygame.image.load(file_name), self.cell_size
            )

        desc = self.desc

        for y in range(0, desc.shape[0]):
            for x in range(0, desc.shape[1]):
                cell = (x * self.cell_size[0], y * self.cell_size[1])
                self.window.blit(self.background_img, cell)
                if desc[y][x] == b"|" and (y == 0 or desc[y - 1][x] != b"|"):
                    self.window.blit(self.median_vert[0], cell)
                elif desc[y][x] == b"|" and (
                    y == desc.shape[0] - 1 or desc[y + 1][x] != b"|"
                ):
                    self.window.blit(self.median_vert[2], cell)
                elif desc[y][x] == b"|":
                    self.window.blit(self.median_vert[1], cell)
                elif desc[y][x] == b"-" and (x == 0 or desc[y][x - 1] != b"-"):
                    self.window.blit(self.median_horiz[0], cell)
                elif desc[y][x] == b"-" and (
                    x == desc.shape[1] - 1 or desc[y][x + 1] != b"-"
                ):
                    self.window.blit(self.median_horiz[2], cell)
                elif desc[y][x] == b"-":
                    self.window.blit(self.median_horiz[1], cell)

        for cell, color in zip(self.locs, self.locs_colors):
            color_cell = pygame.Surface(self.cell_size)
            color_cell.set_alpha(128)
            color_cell.fill(color)
            loc = self.get_surf_loc(cell)
            self.window.blit(color_cell, (loc[0], loc[1] + 10))

        taxi_row, taxi_col, pass_idx, dest_idx = self.decode(self.s)

        if pass_idx < 4:
            self.window.blit(self.passenger_img, self.get_surf_loc(self.locs[pass_idx]))

        if self.lastaction in [0, 1, 2, 3]:
            self.taxi_orientation = self.lastaction
        dest_loc = self.get_surf_loc(self.locs[dest_idx])
        taxi_location = self.get_surf_loc((taxi_row, taxi_col))

        if dest_loc[1] <= taxi_location[1]:
            self.window.blit(
                self.destination_img,
                (dest_loc[0], dest_loc[1] - self.cell_size[1] // 2),
            )
            self.window.blit(self.taxi_imgs[self.taxi_orientation], taxi_location)
        else:  # change blit order for overlapping appearance
            self.window.blit(self.taxi_imgs[self.taxi_orientation], taxi_location)
            self.window.blit(
                self.destination_img,
                (dest_loc[0], dest_loc[1] - self.cell_size[1] // 2),
            )

        if mode == "human":
            pygame.event.pump()
            pygame.display.update()
            self.clock.tick(self.metadata["render_fps"])
        elif mode == "rgb_array":
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(self.window)), axes=(1, 0, 2)
            )

    def get_surf_loc(self, map_loc):
        return (map_loc[1] * 2 + 1) * self.cell_size[0], (
            map_loc[0] + 1
        ) * self.cell_size[1]

    def _render_text(self):
        desc = self.desc.copy().tolist()
        outfile = StringIO()

        out = [[c.decode("utf-8") for c in line] for line in desc]
        taxi_row, taxi_col, pass_idx, dest_idx = self.decode(self.s)

        def ul(x):
            return "_" if x == " " else x

        if pass_idx < 4:
            out[1 + taxi_row][2 * taxi_col + 1] = utils.colorize(
                out[1 + taxi_row][2 * taxi_col + 1], "yellow", highlight=True
            )
            pi, pj = self.locs[pass_idx]
            out[1 + pi][2 * pj + 1] = utils.colorize(
                out[1 + pi][2 * pj + 1], "blue", bold=True
            )
        else:  # passenger in taxi
            out[1 + taxi_row][2 * taxi_col + 1] = utils.colorize(
                ul(out[1 + taxi_row][2 * taxi_col + 1]), "green", highlight=True
            )

        di, dj = self.locs[dest_idx]
        out[1 + di][2 * dj + 1] = utils.colorize(out[1 + di][2 * dj + 1], "magenta")
        outfile.write("\n".join(["".join(row) for row in out]) + "\n")
        if self.lastaction is not None:
            outfile.write(
                f"  ({['South', 'North', 'East', 'West', 'Pickup', 'Dropoff'][self.lastaction]})\n"
            )
        else:
            outfile.write("\n")

        with closing(outfile):
            return outfile.getvalue()

    def close(self):
        if self.window is not None:
            import pygame

            pygame.display.quit()
            pygame.quit()

# Training Strategy(Policy Gradient)

In [5]:
import numpy as np
import sys
from pathlib import Path

In [6]:
class PolicyGradientAgentOptimized:

    def __init__(self, n_states, n_actions,
                 learning_rate=0.01,
                 value_lr=0.1,
                 lr_decay=0.9999,
                 lr_min=0.0001,
                 discount_factor=0.99,
                 entropy_coef=0.01):
        self.n_states = n_states
        self.n_actions = n_actions
        self.lr = learning_rate
        self.lr_init = learning_rate
        self.value_lr = value_lr
        self.lr_decay = lr_decay
        self.lr_min = lr_min
        self.gamma = discount_factor
        self.entropy_coef = entropy_coef

        # policy parameters
        self.theta = np.zeros((n_states, n_actions))

        # Value function
        self.V = np.zeros(n_states)

        # statistics information
        self.episode_count = 0

    def get_policy(self, state):
        """calculate action probability distribution (softmax)"""
        theta_state = self.theta[state] - np.max(self.theta[state])
        exp_theta = np.exp(theta_state)
        return exp_theta / np.sum(exp_theta)

    def choose_action(self, state):
        """sample action according to policy"""
        policy = self.get_policy(state)
        return np.random.choice(self.n_actions, p=policy)

    def update(self, episode_history):
        """
        update policy using Advantage and Entropy
        """
        if len(episode_history) == 0:
            return

        # 1. calculate return G_t
        returns = []
        G = 0
        for state, action, reward in reversed(episode_history):
            G = reward + self.gamma * G
            returns.insert(0, G)
        returns = np.array(returns)

        # 2. update Value function
        for t, (state, action, reward) in enumerate(episode_history):
            td_error = returns[t] - self.V[state]
            self.V[state] += self.value_lr * td_error

        # 3. calculate Advantage
        advantages = []
        for t, (state, action, reward) in enumerate(episode_history):
            advantage = returns[t] - self.V[state]
            advantages.append(advantage)

        # 4. standardize Advantage
        advantages = np.array(advantages)
        if len(advantages) > 1:
            advantages = (advantages - np.mean(advantages)) / (np.std(advantages) + 1e-9)

        # 5. update policy parameters
        for t, (state, action, reward) in enumerate(episode_history):
            policy = self.get_policy(state)

            # Policy gradient
            grad = np.zeros(self.n_actions)
            grad[action] = 1.0
            grad -= policy

            # Entropy gradient
            entropy_grad = -np.log(policy + 1e-9) - 1

            # combine update
            total_grad = (advantages[t] * grad +
                         self.entropy_coef * entropy_grad)

            self.theta[state] += self.lr * total_grad

        # 6. decay learning rate
        self.decay_learning_rate()
        self.episode_count += 1

    def decay_learning_rate(self):
        """gradually decrease learning rate"""
        self.lr = max(self.lr_min, self.lr * self.lr_decay)

    @property
    def Q(self):
        """compatible test function"""
        return self.theta

# Reward Wrapper

In [18]:
class TaxiRewardWrapper(gym.Wrapper): # TODO
    """
    Reward wrapper for Taxi environment
    Allows customizing reward values without modifying the original environment
    Also synchronizes the underlying env's reward attributes so code that
    compares against `env.reward_delivery` etc. sees the customized values.
    """

    def __init__(self, env, reward_step=-5, reward_delivery=20, reward_illegal=-1):
        super().__init__(env)
        self.custom_reward_step = reward_step
        self.custom_reward_delivery = reward_delivery
        self.custom_reward_illegal = reward_illegal

        # Mirror custom values onto the wrapped env so internal transitions
        # that reference env.reward_* return values consistent with the wrapper.
        try:
            # Only set attributes if the wrapped env exposes them
            setattr(self.env, 'reward_step', reward_step)
            setattr(self.env, 'reward_delivery', reward_delivery)
            setattr(self.env, 'reward_illegal', reward_illegal)
        except Exception:
            # Be conservative: if the wrapped env has different API, ignore.
            pass

        # Expose the custom delivery on the wrapper itself for convenience
        self.reward_delivery = self.custom_reward_delivery

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)

        # Replace original rewards with custom rewards based on reward type
        # (kept for backward compatibility in case the underlying env doesn't
        # match the attributes we set above)
        if reward == getattr(self.env, 'reward_delivery', None):
            reward = self.custom_reward_delivery
        elif reward == getattr(self.env, 'reward_illegal', None):
            reward = self.custom_reward_illegal
        elif reward == getattr(self.env, 'reward_step', None):
            reward = self.custom_reward_step

        return obs, reward, terminated, truncated, info

# Training

In [8]:
def train(n_episodes=50000, max_steps=200,
          seed_start=0, seed_end=40000, verbose=True,
          reward_step=-5, reward_delivery=20, reward_illegal=-1):
    """train optimized Policy Gradient Agent"""

    # Wrap environment with custom rewards
    env = TaxiRewardWrapper(
        TaxiEnv(),
        reward_step=reward_step,
        reward_delivery=reward_delivery,
        reward_illegal=reward_illegal
    )

    agent = PolicyGradientAgentOptimized(
        n_states=env.observation_space.n,
        n_actions=env.action_space.n,
        learning_rate=0.01,      # TODO
        value_lr=0.1,         # TODO
        lr_decay=0.99999,       # TODO
        discount_factor=0.99,     # TODO
        entropy_coef=0.1        # TODO
    )

    episode_rewards = []
    success_count = 0

    if verbose:
        print(f"training episodes: {n_episodes}")
        print("=" * 70)

    for episode in range(n_episodes):
        state, info = env.reset(seed=seed_start + episode % seed_end)

        episode_history = []
        total_reward = 0

        for step in range(max_steps):
            action = agent.choose_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            episode_history.append((state, action, reward))

            total_reward += reward
            state = next_state

            if done:
                if terminated and reward == reward_delivery:
                    success_count += 1
                break

        agent.update(episode_history)
        episode_rewards.append(total_reward)

        if verbose and (episode + 1) % 1000 == 0:
            avg_reward = np.mean(episode_rewards[-100:])
            success_rate = success_count / min(1000, episode + 1)

            print(f"episode {episode + 1}/{n_episodes} | "
                  f"avg reward: {avg_reward:.2f} | "
                  f"success rate: {success_rate:.1%} | "
                  f"learning rate: {agent.lr:.6f}")

            if episode >= 99:
                success_count = 0

    if verbose:
        print("=" * 70)
        print("Training completed!")

    env.close()
    return agent, episode_rewards

# Testing

In [9]:
def test(model_filename, n_episodes=100, seed_start=42, verbose=True,
         reward_step=-5, reward_delivery=20, reward_illegal=-1):
    """test agent"""

    # Load model for testing
    print(f"\nload model")

    # Wrap environment with custom rewards
    env = TaxiRewardWrapper(
        TaxiEnv(),
        reward_step=reward_step,
        reward_delivery=reward_delivery,
        reward_illegal=reward_illegal
    )

    agent = PolicyGradientAgentOptimized(
    n_states=env.observation_space.n,
    n_actions=env.action_space.n
    )
    agent.theta = np.load(model_filename)

    rewards = []
    steps_list = []
    successes = 0

    for episode in range(n_episodes):
        state, info = env.reset(seed=seed_start + episode)
        episode_reward = 0
        step_count = 0

        for step in range(200):
            # use deterministic policy during testing
            action = np.argmax(agent.theta[state])

            next_state, reward, terminated, truncated, info = env.step(action)

            episode_reward += reward
            step_count += 1

            if terminated or truncated:
                if reward == reward_delivery:
                    successes += 1
                break

            state = next_state

        rewards.append(episode_reward)
        steps_list.append(step_count)

    avg_reward = np.mean(rewards)
    avg_steps = np.mean(steps_list)
    success_rate = successes / n_episodes

    # calculate evaluation score (success rate 20%, steps 80%)
    normalized_steps = avg_steps / 200
    step_score = 1 - normalized_steps
    evaluation_score = success_rate * 0.2 + step_score * 0.8

    if verbose:
        print(f"\ntest result (seed {seed_start}-{seed_start+n_episodes-1}):")
        print("=" * 70)
        print(f"   avg reward: {avg_reward:.2f}")
        print(f"   avg steps: {avg_steps:.2f}")
        print(f"   success rate: {success_rate:.1%}")
        print(f"   evaluation score: {evaluation_score:.4f} ({evaluation_score*100:.2f}%)")
        print("=" * 70)

    env.close()

    return {
        'avg_reward': avg_reward,
        'avg_steps': avg_steps,
        'success_rate': success_rate,
        'evaluation_score': evaluation_score,
    }

# Main

In [10]:
if __name__ == "__main__":
    print("=" * 70)

    # Set reward parameters
    REWARD_STEP = -1
    REWARD_DELIVERY = 20
    REWARD_ILLEGAL = -10

    # train
    agent, rewards = train(
        n_episodes=50000,
        verbose=True,
        reward_step=REWARD_STEP,
        reward_delivery=REWARD_DELIVERY,
        reward_illegal=REWARD_ILLEGAL
    )

    # save model
    model_filename = 'policy_gradient_optimized.npy'
    np.save(model_filename, agent.theta)
    print(f"\nmodel saved to {model_filename}")

    # test
    test(
        model_filename,
        n_episodes=1000,
        seed_start=420000,
        verbose=True,
        reward_step=REWARD_STEP,
        reward_delivery=REWARD_DELIVERY,
        reward_illegal=REWARD_ILLEGAL
    )


training episodes: 50000
episode 1000/50000 | avg reward: -752.78 | success rate: 4.6% | learning rate: 0.009900
episode 2000/50000 | avg reward: -730.74 | success rate: 7.5% | learning rate: 0.009802
episode 3000/50000 | avg reward: -679.68 | success rate: 9.3% | learning rate: 0.009704
episode 4000/50000 | avg reward: -690.49 | success rate: 9.9% | learning rate: 0.009608
episode 5000/50000 | avg reward: -690.37 | success rate: 14.1% | learning rate: 0.009512
episode 6000/50000 | avg reward: -643.50 | success rate: 18.4% | learning rate: 0.009418
episode 7000/50000 | avg reward: -601.08 | success rate: 25.5% | learning rate: 0.009324
episode 8000/50000 | avg reward: -617.31 | success rate: 26.6% | learning rate: 0.009231
episode 9000/50000 | avg reward: -609.17 | success rate: 30.1% | learning rate: 0.009139
episode 10000/50000 | avg reward: -606.84 | success rate: 33.8% | learning rate: 0.009048
episode 11000/50000 | avg reward: -571.50 | success rate: 38.2% | learning rate: 0.00895

In [11]:
reward_combinations = [
    {'REWARD_STEP': -1, 'REWARD_DELIVERY': 20, 'REWARD_ILLEGAL': -10}, # Original combination
    {'REWARD_STEP': -5, 'REWARD_DELIVERY': 20, 'REWARD_ILLEGAL': -1}, # Higher penalty for steps, lower penalty for illegal actions
    {'REWARD_STEP': -1, 'REWARD_DELIVERY': 10, 'REWARD_ILLEGAL': -20}, # Lower reward for delivery, higher penalty for illegal actions
    {'REWARD_STEP': -3, 'REWARD_DELIVERY': 25, 'REWARD_ILLEGAL': -5}, # Moderate penalties, higher reward for delivery
]

In [12]:
evaluation_results = []

for rewards in reward_combinations:
    REWARD_STEP = rewards['REWARD_STEP']
    REWARD_DELIVERY = rewards['REWARD_DELIVERY']
    REWARD_ILLEGAL = rewards['REWARD_ILLEGAL']

    print("=" * 70)
    print(f"Training with REWARD_STEP={REWARD_STEP}, REWARD_DELIVERY={REWARD_DELIVERY}, REWARD_ILLEGAL={REWARD_ILLEGAL}")
    print("=" * 70)

    agent, episode_rewards = train(
        n_episodes=50000,
        verbose=True,
        reward_step=REWARD_STEP,
        reward_delivery=REWARD_DELIVERY,
        reward_illegal=REWARD_ILLEGAL
    )

    model_filename = f'policy_gradient_step{REWARD_STEP}_delivery{REWARD_DELIVERY}_illegal{REWARD_ILLEGAL}.npy'
    np.save(model_filename, agent.theta)
    print(f"\nmodel saved to {model_filename}")

Training with REWARD_STEP=-1, REWARD_DELIVERY=20, REWARD_ILLEGAL=-10
training episodes: 50000
episode 1000/50000 | avg reward: -745.37 | success rate: 5.3% | learning rate: 0.009900
episode 2000/50000 | avg reward: -729.94 | success rate: 8.2% | learning rate: 0.009802
episode 3000/50000 | avg reward: -714.77 | success rate: 10.5% | learning rate: 0.009704
episode 4000/50000 | avg reward: -640.69 | success rate: 15.4% | learning rate: 0.009608
episode 5000/50000 | avg reward: -652.02 | success rate: 18.7% | learning rate: 0.009512
episode 6000/50000 | avg reward: -619.32 | success rate: 21.6% | learning rate: 0.009418
episode 7000/50000 | avg reward: -609.55 | success rate: 28.4% | learning rate: 0.009324
episode 8000/50000 | avg reward: -621.53 | success rate: 28.6% | learning rate: 0.009231
episode 9000/50000 | avg reward: -633.55 | success rate: 32.6% | learning rate: 0.009139
episode 10000/50000 | avg reward: -553.06 | success rate: 37.0% | learning rate: 0.009048
episode 11000/500

In [16]:
# # test
# test(
#     model_filename,
#     n_episodes=1000,
#     seed_start=420000,
#     verbose=True,
#     reward_step=REWARD_STEP,
#     reward_delivery=REWARD_DELIVERY,
#     reward_illegal=REWARD_ILLEGAL
# )


import os
for file in os.listdir("./"):
    if file.endswith(".npy"):
        model_filename = os.path.join("", file)
        print(os.path.join("./", model_filename))
        test(
            model_filename,
            n_episodes=1000,
            seed_start=420000,
            verbose=True,
            reward_step=REWARD_STEP,
            reward_delivery=REWARD_DELIVERY,
            reward_illegal=REWARD_ILLEGAL
        )

./policy_gradient_step-1_delivery20_illegal-10.npy

load model

test result (seed 420000-420999):
   avg reward: -415.52
   avg steps: 141.49
   success rate: 31.9%
   evaluation score: 0.2979 (29.79%)
./policy_gradient_step-3_delivery25_illegal-5.npy

load model

test result (seed 420000-420999):
   avg reward: -455.33
   avg steps: 146.12
   success rate: 29.0%
   evaluation score: 0.2735 (27.35%)
./policy_gradient_step-5_delivery20_illegal-1.npy

load model

test result (seed 420000-420999):
   avg reward: -688.70
   avg steps: 160.49
   success rate: 21.1%
   evaluation score: 0.2003 (20.03%)
./policy_gradient_step-1_delivery10_illegal-20.npy

load model

test result (seed 420000-420999):
   avg reward: -394.47
   avg steps: 134.79
   success rate: 35.4%
   evaluation score: 0.3316 (33.16%)
./policy_gradient_optimized.npy

load model

test result (seed 420000-420999):
   avg reward: -421.62
   avg steps: 143.40
   success rate: 30.6%
   evaluation score: 0.2876 (28.76%)


In [ ]:
class RewardCloserWrapper(gym.Wrapper):
    """
    Reward shaping wrapper that gives a small positive reward when the taxi moves
    closer to the current passenger (if not picked up) or to the destination
    (if passenger is onboard). This keeps `TaxiEnv` unchanged and is toggleable.
    """

    def __init__(self, env, shaping_reward=0.5):
        super().__init__(env)
        self.shaping_reward = shaping_reward
        self.prev_state = None

    def _manhattan(self, a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)

        try:
            if self.prev_state is None:
                self.prev_state = obs

            prev_decoded = self.env.decode(self.prev_state)
            curr_decoded = self.env.decode(obs)
            prev_row, prev_col, prev_pass_idx, prev_dest = prev_decoded
            curr_row, curr_col, curr_pass_idx, curr_dest = curr_decoded

            if prev_pass_idx < 4:
                target_prev = self.env.locs[prev_pass_idx]
            else:
                target_prev = self.env.locs[prev_dest]

            if curr_pass_idx < 4:
                target_curr = self.env.locs[curr_pass_idx]
            else:
                target_curr = self.env.locs[curr_dest]

            prev_dist = self._manhattan((prev_row, prev_col), target_prev)
            curr_dist = self._manhattan((curr_row, curr_col), target_curr)

            if curr_dist < prev_dist and not terminated:
                reward = reward + self.shaping_reward

        except Exception:
            pass

        self.prev_state = obs
        return obs, reward, terminated, truncated, info

In [13]:
# Quick shaping experiment: short train + eval using the wrapper
# NOTE: this cell runs a compact training loop (uses the PolicyGradientAgentOptimized defined earlier)

n_episodes = 50000
max_steps = 200
shaping_reward = 0.5

# Build wrapped environment for training
env = RewardCloserWrapper(
    TaxiRewardWrapper(TaxiEnv(), reward_step=-1, reward_delivery=20, reward_illegal=-10),
    shaping_reward=shaping_reward,
)

agent = PolicyGradientAgentOptimized(
    n_states=env.observation_space.n,
    n_actions=env.action_space.n,
    learning_rate=0.01,
    value_lr=0.1,
    lr_decay=0.99999,
    discount_factor=0.99,
    entropy_coef=0.1,
)

episode_rewards = []

for episode in range(n_episodes):
    state, info = env.reset(seed=episode)
    total_reward = 0
    episode_history = []

    for step in range(max_steps):
        action = agent.choose_action(state)
        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        episode_history.append((state, action, reward))
        total_reward += reward
        state = next_state

        if done:
            break

    agent.update(episode_history)
    episode_rewards.append(total_reward)

    if (episode + 1) % 500 == 0:
        print(f"trained {episode+1}/{n_episodes} | avg recent reward: {np.mean(episode_rewards[-100:]):.2f}")

# Save model
model_filename = 'policy_gradient_shaped.npy'
np.save(model_filename, agent.theta)
print(f"saved model to {model_filename}")

# Quick evaluation using the same wrapper
n_test = 200
rewards = []
steps_list = []
successes = 0

for episode in range(n_test):
    state, info = env.reset(seed=1000 + episode)
    episode_reward = 0
    step_count = 0

    for step in range(200):
        action = np.argmax(agent.theta[state])
        next_state, reward, terminated, truncated, info = env.step(action)
        episode_reward += reward
        step_count += 1
        if terminated or truncated:
            if reward == env.env.custom_reward_delivery:
                successes += 1
            break
        state = next_state

    rewards.append(episode_reward)
    steps_list.append(step_count)

avg_reward = np.mean(rewards)
avg_steps = np.mean(steps_list)
success_rate = successes / n_test
normalized_steps = avg_steps / 200
step_score = 1 - normalized_steps
evaluation_score = success_rate * 0.2 + step_score * 0.8

print("\nshaped test result:")
print("="*60)
print(f"   avg reward: {avg_reward:.2f}")
print(f"   avg steps: {avg_steps:.2f}")
print(f"   success rate: {success_rate:.1%}")
print(f"   evaluation score: {evaluation_score:.4f} ({evaluation_score*100:.2f}%)")
print("="*60)


trained 500/50000 | avg recent reward: -759.52
trained 1000/50000 | avg recent reward: -737.78
trained 1000/50000 | avg recent reward: -737.78
trained 1500/50000 | avg recent reward: -717.49
trained 1500/50000 | avg recent reward: -717.49
trained 2000/50000 | avg recent reward: -725.37
trained 2000/50000 | avg recent reward: -725.37
trained 2500/50000 | avg recent reward: -681.19
trained 2500/50000 | avg recent reward: -681.19
trained 3000/50000 | avg recent reward: -670.33
trained 3000/50000 | avg recent reward: -670.33
trained 3500/50000 | avg recent reward: -677.06
trained 3500/50000 | avg recent reward: -677.06
trained 4000/50000 | avg recent reward: -691.62
trained 4000/50000 | avg recent reward: -691.62
trained 4500/50000 | avg recent reward: -660.64
trained 4500/50000 | avg recent reward: -660.64
trained 5000/50000 | avg recent reward: -654.75
trained 5000/50000 | avg recent reward: -654.75
trained 5500/50000 | avg recent reward: -629.75
trained 5500/50000 | avg recent reward: -

In [19]:
# Compact, masked tabular Q-learning baseline (fast)
# Defines masked helpers and Q-learning train/test functions.
import numpy as np


def masked_random_choice(mask):
    """Return a random action index among allowed actions in mask (0/1 mask)."""
    mask = np.asarray(mask, dtype=np.int8)
    allowed = np.nonzero(mask)[0]
    if allowed.size == 0:
        return int(np.random.randint(0, len(mask)))
    return int(np.random.choice(allowed))


def masked_argmax(Q_row, mask):
    """Masked argmax: choose argmax among allowed actions; break ties by smallest index."""
    mask = np.asarray(mask, dtype=np.int8)
    if mask.sum() == 0:
        return int(np.argmax(Q_row))
    scores = np.where(mask, Q_row, -np.inf)
    return int(np.argmax(scores))


def train_q_learning(env, n_episodes=2000, alpha=0.1, gamma=0.99, epsilon=0.1, max_steps=200, seed_start=0, verbose=True):
    """Train a tabular Q-learning agent using action masks from the env.
    Returns Q-table (n_states x n_actions) and training stats.
    """
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    Q = np.zeros((n_states, n_actions), dtype=float)

    episode_rewards = []
    steps_per_episode = []
    successes = 0

    for ep in range(n_episodes):
        state, info = env.reset(seed=seed_start + ep)
        total_reward = 0.0
        for t in range(max_steps):
            mask = info.get('action_mask', np.ones(n_actions, dtype=int))
            if np.random.random() < epsilon:
                action = masked_random_choice(mask)
            else:
                action = masked_argmax(Q[state], mask)

            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            next_mask = info.get('action_mask', np.ones(n_actions, dtype=int))
            if next_mask.sum() == 0:
                next_max = np.max(Q[next_state])
            else:
                next_max = np.max(np.where(next_mask, Q[next_state], -np.inf))
                if np.isneginf(next_max):
                    next_max = np.max(Q[next_state])

            td_target = reward + gamma * next_max * (0 if done else 1)
            td_error = td_target - Q[state, action]
            Q[state, action] += alpha * td_error

            total_reward += reward
            state = next_state

            if done:
                if terminated and reward == env.reward_delivery:
                    successes += 1
                break

        episode_rewards.append(total_reward)
        steps_per_episode.append(t + 1)

        if verbose and (ep + 1) % 500 == 0:
            print(f"Episode {ep+1}/{n_episodes} | avg reward (last 100): {np.mean(episode_rewards[-100:]):.2f} | avg steps (last 100): {np.mean(steps_per_episode[-100:]):.1f}")

    stats = {
        'episode_rewards': episode_rewards,
        'steps_per_episode': steps_per_episode,
        'successes': successes,
    }
    return Q, stats


def test_q_learning(Q, env, n_episodes=200, seed_start=0, max_steps=200, verbose=True):
    """Evaluate a Q-table deterministically using masked argmax. Returns metrics dict."""
    n_actions = env.action_space.n
    rewards = []
    steps = []
    successes = 0

    for ep in range(n_episodes):
        state, info = env.reset(seed=seed_start + ep)
        ep_reward = 0.0
        for t in range(max_steps):
            mask = info.get('action_mask', np.ones(n_actions, dtype=int))
            action = masked_argmax(Q[state], mask)
            next_state, reward, terminated, truncated, info = env.step(action)
            ep_reward += reward
            if terminated or truncated:
                if terminated and reward == env.reward_delivery:
                    successes += 1
                break
            state = next_state
        rewards.append(ep_reward)
        steps.append(t + 1)

    avg_reward = np.mean(rewards)
    avg_steps = np.mean(steps)
    success_rate = successes / n_episodes
    normalized_steps = avg_steps / max_steps
    step_score = 1 - normalized_steps
    evaluation_score = success_rate * 0.2 + step_score * 0.8

    metrics = {
        'avg_reward': avg_reward,
        'avg_steps': avg_steps,
        'success_rate': success_rate,
        'evaluation_score': evaluation_score,
    }
    if verbose:
        print("\nQ-learning test result:")
        print("="*60)
        print(f"  avg reward: {avg_reward:.2f}")
        print(f"  avg steps: {avg_steps:.2f}")
        print(f"  success rate: {success_rate:.1%}")
        print(f"  evaluation score: {evaluation_score:.4f} ({evaluation_score*100:.2f}%)")
        print("="*60)
    return metrics


In [21]:
# Run a short Q-learning baseline using the Taxi env and masked actions
# Configure these hyperparameters as needed

from pathlib import Path
import numpy as np

n_train = 50000
alpha = 0.1
gamma = 0.99
epsilon = 0.1
max_steps = 200

# Use the project's reward wrapper and environment
env = TaxiRewardWrapper(TaxiEnv(), reward_step=-1, reward_delivery=20, reward_illegal=-10)

print('Training Q-learning...')
Q, stats = train_q_learning(env, n_episodes=n_train, alpha=alpha, gamma=gamma, epsilon=epsilon, max_steps=max_steps, verbose=True)

model_path = Path('./q_learning_baseline.npy')
np.save(model_path, Q)
print(f'Saved Q-table to {model_path}')

print('Evaluating learned Q-table...')
metrics = test_q_learning(Q, env, n_episodes=500, seed_start=10000, max_steps=max_steps, verbose=True)

print('\nDone. Metrics:')
print(metrics)


Training Q-learning...
Episode 500/50000 | avg reward (last 100): -51.13 | avg steps (last 100): 71.3
Episode 1000/50000 | avg reward (last 100): -22.86 | avg steps (last 100): 43.6
Episode 1500/50000 | avg reward (last 100): -4.17 | avg steps (last 100): 25.2
Episode 2000/50000 | avg reward (last 100): -2.21 | avg steps (last 100): 23.2
Episode 2500/50000 | avg reward (last 100): -1.82 | avg steps (last 100): 22.8
Episode 3000/50000 | avg reward (last 100): 0.42 | avg steps (last 100): 20.6
Episode 3500/50000 | avg reward (last 100): 1.25 | avg steps (last 100): 19.8
Episode 4000/50000 | avg reward (last 100): 0.26 | avg steps (last 100): 20.7
Episode 4500/50000 | avg reward (last 100): 0.27 | avg steps (last 100): 20.7
Episode 5000/50000 | avg reward (last 100): -0.32 | avg steps (last 100): 21.3
Episode 5500/50000 | avg reward (last 100): 0.83 | avg steps (last 100): 20.2
Episode 6000/50000 | avg reward (last 100): 0.69 | avg steps (last 100): 20.3
Episode 6500/50000 | avg reward (l

In [25]:
print('Evaluating learned Q-table...')
metrics = test_q_learning(Q, env, n_episodes=3000, seed_start=49, max_steps=max_steps, verbose=True)

print('\nDone. Metrics:')
print(metrics)

Evaluating learned Q-table...

Q-learning test result:
  avg reward: 1.47
  avg steps: 19.53
  success rate: 100.0%
  evaluation score: 0.9219 (92.19%)

Done. Metrics:
{'avg_reward': np.float64(1.4743333333333333), 'avg_steps': np.float64(19.525666666666666), 'success_rate': 1.0, 'evaluation_score': np.float64(0.9218973333333333)}


In [27]:
env

<TaxiRewardWrapper<TaxiEnv instance>>

## Action-mask experiment: Policy Gradient vs Q-learning

This cell runs 4 short experiments (quick smoke tests):

- PolicyGradientAgent (unmasked)
- PolicyGradientAgent (masked action selection using `info['action_mask']`)
- Q-learning (unmasked)
- Q-learning (masked — uses the masked helpers already defined)

Results (metrics) are collected into a small pandas DataFrame and saved to
`actionmask_experiment_summary.csv`. Models (.npy) are saved next to the notebook.

Note: these are short runs (n_train small) intended for quick comparisons —
increase `n_train` for stronger baselines.

In [28]:
import pandas as pd
import time

# Helper: sample action from a policy but respect action mask (0/1 array) if provided
def sample_action_from_policy(agent, state, mask=None):
    policy = agent.get_policy(state).astype(float)
    if mask is None:
        return np.random.choice(agent.n_actions, p=policy)
    mask = np.asarray(mask, dtype=int)
    masked = policy * mask
    s = masked.sum()
    if s <= 0 or not np.isfinite(s):
        # fallback: use original policy (renormalized) if mask disables all actions
        policy = policy / policy.sum()
        return np.random.choice(agent.n_actions, p=policy)
    masked = masked / s
    return np.random.choice(agent.n_actions, p=masked)

# Policy Gradient trainer that optionally uses action masks during sampling
def train_policy_gradient_custom(n_episodes=2000, max_steps=200, use_mask=False, reward_step=-1, reward_delivery=20, reward_illegal=-10):
    env = TaxiRewardWrapper(TaxiEnv(), reward_step=reward_step, reward_delivery=reward_delivery, reward_illegal=reward_illegal)
    agent = PolicyGradientAgentOptimized(n_states=env.observation_space.n, n_actions=env.action_space.n, learning_rate=0.01, value_lr=0.1, lr_decay=0.99999, discount_factor=0.99, entropy_coef=0.1)

    episode_rewards = []
    for ep in range(n_episodes):
        state, info = env.reset(seed=ep)
        total_reward = 0.0
        episode_history = []
        for t in range(max_steps):
            if use_mask:
                mask = info.get('action_mask', np.ones(agent.n_actions, dtype=int))
                action = sample_action_from_policy(agent, state, mask)
            else:
                action = agent.choose_action(state)

            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            episode_history.append((state, action, reward))
            total_reward += reward
            state = next_state
            if done:
                break

        agent.update(episode_history)
        episode_rewards.append(total_reward)

    return agent, episode_rewards

# Q-learning trainer (unmasked) — same as train_q_learning but ignores action_mask during selection
def train_q_learning_unmasked(env, n_episodes=2000, alpha=0.1, gamma=0.99, epsilon=0.1, max_steps=200, seed_start=0, verbose=True):
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    Q = np.zeros((n_states, n_actions), dtype=float)
    episode_rewards = []
    steps_per_episode = []
    successes = 0
    for ep in range(n_episodes):
        state, info = env.reset(seed=seed_start + ep)
        total_reward = 0.0
        for t in range(max_steps):
            # ignore mask: epsilon-greedy across all actions
            if np.random.random() < epsilon:
                action = int(np.random.randint(0, n_actions))
            else:
                action = int(np.argmax(Q[state]))

            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            next_max = np.max(Q[next_state])
            td_target = reward + gamma * next_max * (0 if done else 1)
            td_error = td_target - Q[state, action]
            Q[state, action] += alpha * td_error

            total_reward += reward
            state = next_state
            if done:
                if terminated and reward == env.reward_delivery:
                    successes += 1
                break

        episode_rewards.append(total_reward)
        steps_per_episode.append(t + 1)

    stats = {'episode_rewards': episode_rewards, 'steps_per_episode': steps_per_episode, 'successes': successes}
    return Q, stats

# Evaluation helpers
def evaluate_policy_agent(agent, env, n_episodes=200, use_mask=False, max_steps=200):
    rewards = []
    steps = []
    successes = 0
    for ep in range(n_episodes):
        state, info = env.reset(seed=ep)
        ep_reward = 0.0
        for t in range(max_steps):
            if use_mask:
                mask = info.get('action_mask', np.ones(agent.n_actions, dtype=int))
                action = masked_argmax(agent.theta[state], mask)
            else:
                action = int(np.argmax(agent.theta[state]))
            next_state, reward, terminated, truncated, info = env.step(action)
            ep_reward += reward
            if terminated or truncated:
                if terminated and reward == env.reward_delivery:
                    successes += 1
                break
            state = next_state
        rewards.append(ep_reward)
        steps.append(t+1)
    avg_reward = np.mean(rewards)
    avg_steps = np.mean(steps)
    success_rate = successes / n_episodes
    normalized_steps = avg_steps / max_steps
    step_score = 1 - normalized_steps
    evaluation_score = success_rate * 0.2 + step_score * 0.8
    return {'avg_reward': avg_reward, 'avg_steps': avg_steps, 'success_rate': success_rate, 'evaluation_score': evaluation_score}

# Run the 4 short experiments and collect results
results = []
n_pg = 2000
n_q = 2000
print('Starting experiments — this will take a short while...')
start_all = time.time()
# 1) PolicyGradient unmasked
agent_pg_unmasked, rewards_pg_unmasked = train_policy_gradient_custom(n_episodes=n_pg, use_mask=False)
np.save('pg_unmasked.npy', agent_pg_unmasked.theta)
env_eval = TaxiRewardWrapper(TaxiEnv(), reward_step=-1, reward_delivery=20, reward_illegal=-10)
metrics = evaluate_policy_agent(agent_pg_unmasked, env_eval, n_episodes=200, use_mask=False)
metrics.update({'agent':'PolicyGradient','mode':'unmasked','n_train':n_pg})
results.append(metrics)
print('PG unmasked done')
# 2) PolicyGradient masked
agent_pg_masked, rewards_pg_masked = train_policy_gradient_custom(n_episodes=n_pg, use_mask=True)
np.save('pg_masked.npy', agent_pg_masked.theta)
env_eval = TaxiRewardWrapper(TaxiEnv(), reward_step=-1, reward_delivery=20, reward_illegal=-10)
metrics = evaluate_policy_agent(agent_pg_masked, env_eval, n_episodes=200, use_mask=True)
metrics.update({'agent':'PolicyGradient','mode':'masked','n_train':n_pg})
results.append(metrics)
print('PG masked done')
# 3) Q-learning unmasked
env_q = TaxiRewardWrapper(TaxiEnv(), reward_step=-1, reward_delivery=20, reward_illegal=-10)
Q_unmasked, stats_unmasked = train_q_learning_unmasked(env_q, n_episodes=n_q, alpha=0.1, gamma=0.99, epsilon=0.1, max_steps=200, verbose=True)
np.save('q_unmasked.npy', Q_unmasked)
metrics = test_q_learning(Q_unmasked, env_q, n_episodes=200, seed_start=10000, max_steps=200, verbose=False)
metrics.update({'agent':'Q-learning','mode':'unmasked','n_train':n_q})
results.append(metrics)
print('Q unmasked done')
# 4) Q-learning masked (uses existing train_q_learning which is masked-aware)
env_q2 = TaxiRewardWrapper(TaxiEnv(), reward_step=-1, reward_delivery=20, reward_illegal=-10)
Q_masked, stats_masked = train_q_learning(env_q2, n_episodes=n_q, alpha=0.1, gamma=0.99, epsilon=0.1, max_steps=200, verbose=True)
np.save('q_masked.npy', Q_masked)
metrics = test_q_learning(Q_masked, env_q2, n_episodes=200, seed_start=10000, max_steps=200, verbose=False)
metrics.update({'agent':'Q-learning','mode':'masked','n_train':n_q})
results.append(metrics)
print('Q masked done')

df = pd.DataFrame(results)
csv_out = 'actionmask_experiment_summary.csv'
df.to_csv(csv_out, index=False)
print(f'Experiment finished in {time.time()-start_all:.1f}s — summary saved to {csv_out}')
print(df)


Starting experiments — this will take a short while...
PG unmasked done
PG masked done
Q unmasked done
Episode 500/2000 | avg reward (last 100): -61.99 | avg steps (last 100): 80.9
Episode 1000/2000 | avg reward (last 100): -24.65 | avg steps (last 100): 45.6
Episode 1500/2000 | avg reward (last 100): -6.41 | avg steps (last 100): 27.4
Episode 2000/2000 | avg reward (last 100): -3.34 | avg steps (last 100): 24.3
Q masked done
Experiment finished in 58.6s — summary saved to actionmask_experiment_summary.csv
   avg_reward  avg_steps  success_rate  evaluation_score           agent  \
0    -314.570    190.130         0.055           0.05048  PolicyGradient   
1     -79.380     92.715         0.635           0.55614  PolicyGradient   
2     -83.735     96.545         0.610           0.53582      Q-learning   
3     -17.465     36.890         0.925           0.83744      Q-learning   

       mode  n_train  
0  unmasked     2000  
1    masked     2000  
2  unmasked     2000  
3    masked    